<a href="https://colab.research.google.com/github/Tanisha2626/Harasment_level_detection/blob/main/Harasment_Level_detection_Bengali_(2)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/huggingface/transformers
!pip install /content/transformers/
!pip install sentencepiece

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

### TPU set-up

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

### Downloading Datasets

In [ ]:
!git clone https://github.com/Tanisha2626/Harasment_level_detection

In [ ]:
raw_train_df =  pd.read_csv("Harasment_level_detection/trac2_iben_train.csv")
raw_train_df['split'] = 'train'
print(raw_train_df.columns)
print(raw_train_df['Sub-task A'].value_counts())
print(raw_train_df['Sub-task B'].value_counts())
print(f"Size of 'train' split: {len(raw_train_df)}")

In [ ]:
raw_dev_df =  pd.read_csv("Harasment_level_detection/trac2_iben_dev.csv")
raw_dev_df['split'] = 'dev'
print(raw_dev_df.columns)
print(raw_dev_df['Sub-task A'].value_counts())
print(raw_dev_df['Sub-task B'].value_counts())
print(f"Size of 'dev' split: {len(raw_dev_df)}")

In [ ]:
test_df =  pd.read_csv("Harasment_level_detection/trac2_iben_test.csv")
test_df

In [ ]:
# Concatinate both train and dev dfs together
data_df = pd.concat([raw_dev_df, raw_train_df], ignore_index= True)
data_df

### Samples given per label size

In [ ]:
print(f'Total dev + train size = {len(data_df)}\n')
print(data_df['Sub-task A'].value_counts(),'\n')
print(data_df['Sub-task B'].value_counts(),'\n')

### Label encoder for Sub-task A

In [ ]:
task_a_label_dict = {'NAG':0, 'OAG':1, 'CAG':2}
print(task_a_label_dict)

In [ ]:
data_df_task_a = data_df[['ID','Text','Sub-task A','split']].copy()
data_df_task_a.columns.values[1] = 'text'
data_df_task_a.columns.values[2] = 'label'
data_df_task_a.loc[:,'label'] = data_df_task_a.loc[:,'label'].map(task_a_label_dict)
data_df_task_a

In [ ]:
print("Num samples per class")
print(data_df_task_a.label.value_counts())

print("\nNum samples per split")
print(data_df_task_a.split.value_counts())

print("\nLabel counts in dev split")
print(data_df_task_a[data_df_task_a.split=='dev'].label.value_counts())

print("\nLabel counts in train split")
print(data_df_task_a[data_df_task_a.split=='train'].label.value_counts())

### Label encoder for Sub-task B

In [ ]:
task_b_label_dict = {'NGEN':0, 'GEN':1}
print(task_b_label_dict)

In [ ]:
data_df_task_b = data_df[['ID','Text','Sub-task B','split']].copy()
data_df_task_b.columns.values[1] = 'text'
data_df_task_b.columns.values[2] = 'label'
data_df_task_b.loc[:,'label'] = data_df_task_b.loc[:,'label'].map(task_b_label_dict)
data_df_task_b

In [ ]:
print("Num samples per class")
print(data_df_task_b.label.value_counts())

print("\nNum samples per split")
print(data_df_task_b.split.value_counts())

print("\nLabel counts in dev split")
print(data_df_task_b[data_df_task_b.split=='dev'].label.value_counts())

print("\nLabel counts in train split")
print(data_df_task_b[data_df_task_b.split=='train'].label.value_counts())

# Sub-task B

In [ ]:
#gb = data_df_task_a.groupby('split')
gb = data_df_task_b.groupby('split')
grps = [gb.get_group(x) for x in gb.groups]

In [ ]:
grps[1]

### Model for sub-task b

In [ ]:
def build_model_task_b(transformer, num_classes, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    #cls_token = sequence_output[:, 0, :]
    #out = Dense(num_classes, activation='softmax')(cls_token)
    X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1))(sequence_output)
    X = tf.keras.layers.GlobalMaxPool1D()(X)
    X = tf.keras.layers.Dense(50, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    out = tf.keras.layers.Dense(num_classes, activation='softmax')(X)
    model = Model(inputs=input_word_ids, outputs=out)
    #model.layers[1].trainable = False
    model.compile(Adam(lr=1e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model


### Hyperparameters

In [ ]:
EPOCHS = 2
BATCH_SIZE = 16 #* strategy.num_replicas_in_sync
MAX_LEN = 128
MODEL = 'jplu/tf-xlm-roberta-base'

### Tokenizer

In [ ]:
from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case=True, add_special_tokens=True,max_length=512, pad_to_max_length=True)

In [ ]:
x_train = np.asarray(tokenizer.batch_encode_plus(grps[1]['text'],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])

In [ ]:
x_train.shape

In [ ]:
y_train = grps[1].label

In [ ]:
y_train = np.asarray(y_train, dtype=np.float32)

In [ ]:
x_val = np.asarray(tokenizer.batch_encode_plus(grps[0]['text'],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])

In [ ]:
x_val.shape

In [ ]:
y_val = grps[0].label

In [ ]:
y_val = np.asarray(y_val, dtype=np.float32)

In [ ]:
x_test = np.asarray(tokenizer.batch_encode_plus(test_df['Text'],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

## Training

In [ ]:
# # Train on CPU/GPU

# transformer_layer = TFAutoModel.from_pretrained(MODEL)
# model_task_b = build_model_task_b(transformer_layer,num_classes=2, max_len=MAX_LEN)
# model_task_b.summary()

# n_steps = x_train.shape[0] // BATCH_SIZE
# train_history = model_task_b.fit(
#     train_dataset,
#     steps_per_epoch=n_steps,
#     validation_data=val_dataset,
#     epochs=EPOCHS
# )

In [ ]:
# Train on TPU

with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model_task_b = build_model_task_b(transformer_layer,num_classes=2, max_len=MAX_LEN)
model_task_b.summary()

n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model_task_b.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=val_dataset,
    epochs=EPOCHS
)

In [ ]:
np.argmax(model_task_b.predict(x_test),axis=1)

In [ ]:
predicted= np.argmax(model_task_b.predict(x_val),axis=1)

y_true = np.asarray(y_val)

In [ ]:
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,confusion_matrix

print(precision_score(y_true,predicted,average='macro'))
print(recall_score(y_true,predicted,average='macro'))
print(f1_score(y_true,predicted,average='macro'))
print(accuracy_score(y_true,predicted))
print(confusion_matrix(y_true, predicted))

## Testing function for sub-task b

In [ ]:
import warnings
warnings.filterwarnings("ignore")
def gender_bias(sentence):
  text = [sentence]
  df_input = pd.DataFrame(text)
  x_input = np.asarray(tokenizer.batch_encode_plus(df_input[0],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])
  class_probs = model_task_b.predict(x_input)
  pred_class = np.argmax(class_probs,axis=1)[0]
  gender = list(task_b_label_dict.keys())[pred_class]
  return gender, class_probs[0]

## GUI input for sub-task b

In [ ]:
#@title Is this comment gender targeted?
sentence = "tumi paagol mohila" #@param {type:"string"}
gender, class_probs = gender_bias(sentence)
print("Is it Gender Targeted ? = ",gender)
print("Not Gender Targeted probability =",class_probs[0])
print("Gender Targeted probability =",class_probs[1])

Is it Gender Targeted ? =  GEN
Not Gender Targeted probability = 0.38148755
Gender Targeted probability = 0.6185124


# Sub-Task-A

In [ ]:
gb = data_df_task_a.groupby('split')
grps = [gb.get_group(x) for x in gb.groups]

In [ ]:
x_train = np.asarray(tokenizer.batch_encode_plus(grps[1]['text'],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])

In [ ]:
y_train = grps[1].label

In [ ]:
y_train = np.asarray(y_train, dtype=np.float32)

In [ ]:
x_val = np.asarray(tokenizer.batch_encode_plus(grps[0]['text'],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])

In [ ]:
y_val = grps[0].label

In [ ]:
y_val = np.asarray(y_val, dtype=np.float32)

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

## Model for sub-task A

In [ ]:
def build_model_task_a(transformer, num_classes, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    #cls_token = sequence_output[:, 0, :]
    #out = Dense(num_classes, activation='softmax')(cls_token)
    X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1))(sequence_output)
    X = tf.keras.layers.GlobalMaxPool1D()(X)
    X = tf.keras.layers.Dense(50, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    out = tf.keras.layers.Dense(num_classes, activation='softmax')(X)
    model = Model(inputs=input_word_ids, outputs=out)
    #model.layers[1].trainable = False
    model.compile(Adam(lr=1e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model


## Training

In [ ]:
# # Train on CPU/GPU

# transformer_layer = TFAutoModel.from_pretrained(MODEL)
# model_task_a = build_model_task_a(transformer_layer,num_classes=3, max_len=MAX_LEN)
# model_task_a.summary()

# n_steps = x_train.shape[0] // BATCH_SIZE
# train_history = model_task_a.fit(
#     train_dataset,
#     steps_per_epoch=n_steps,
#     validation_data=val_dataset,
#     epochs=20
# )

In [ ]:
# Train on TPU
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model_task_a = build_model_task_a(transformer_layer, num_classes=3, max_len=MAX_LEN)
model_task_a.summary()

n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model_task_a.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=val_dataset,
    epochs=EPOCHS
)

In [ ]:
predicted= np.argmax(model_task_a.predict(x_val),axis=1)

y_true = np.asarray(y_val)

In [ ]:
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,confusion_matrix

print(precision_score(y_true,predicted,average='macro'))
print(recall_score(y_true,predicted,average='macro'))
print(f1_score(y_true,predicted,average='macro'))
print(accuracy_score(y_true,predicted))
print(confusion_matrix(y_true, predicted))

## Testing function for sub-task A

In [ ]:
import warnings
warnings.filterwarnings("ignore")
def agression(sentence):
  text = [sentence]
  df_input = pd.DataFrame(text)
  x_input = np.asarray(tokenizer.batch_encode_plus(df_input[0],pad_to_max_length=True, max_length=128, return_attention_mask=False)['input_ids'])
  class_probs = model_task_a.predict(x_input)
  pred_class = np.argmax(class_probs,axis=1)[0]
  agression = list(task_a_label_dict.keys())[pred_class]
  return agression, class_probs[0]

## GUI input for sub-task A

In [ ]:
#@title Agression Level
sentence = "Aaami tomaake ekdum jore maarbo" #@param {type:"string"}
agr, class_probs = agression(sentence)
print("Agression level = ",agr)
print("NAG probability =",class_probs[0])
print("OAG probability =",class_probs[1])
print("CAG probability =",class_probs[2])

Agression level =  OAG
NAG probability = 0.29471526
OAG probability = 0.3587213
CAG probability = 0.34656352
